In [5]:
!pip install -r requirements.txt

  Using cached en_core_web_sm-2.2.0-py3-none-any.whl
     |████████████████████████████████| 10.2 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 62.4 MB/s eta 0:00:01
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.1
    Uninstalling thinc-7.4.1:
      Successfully uninstalled thinc-7.4.1
  Attempting uninstall: spacy
    Found existing installation: spacy 2.3.0
    Uninstalling spacy-2.3.0:
      Successfully uninstalled spacy-2.3.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [7]:
import spacy
import en_core_web_sm
nlp=en_core_web_sm.load()
from nltk.tokenize import sent_tokenize, word_tokenize   
import numpy as np 
import matplotlib.pyplot as plt
import scikitplot as skplt
from timeit import default_timer as timer 
import os
import random 
random.seed(100)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import pickle
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full
import gensim
from Utils import *
import dask.dataframe as dd
from dask.distributed import Client
import gc

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
import h2o
h2o.connect()

Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,17 mins 48 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,H2O_from_python_ec2_user_deawsv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.949 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


<H2OConnection to http://localhost:54321, no session>

In [20]:
data= pd.read_csv("Train.csv")

In [21]:
data= data.head(1000)

In [22]:
data2=dd.from_pandas(data, npartitions=2)

data2['clean_text'] = data2['text'].apply(lambda x: data_clean(x), meta=('sentence')).compute(scheduler='processes')

data=data2.compute(num_workers=2)

In [23]:
tfidf_vect = TfidfVectorizer(analyzer='word',max_df=0.75,stop_words='english', ngram_range=(1,1),max_features=300)
tfidf_fit = tfidf_vect.fit_transform(data['clean_text'])
tfidf_array = tfidf_fit.toarray()

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,6), max_features=3000,max_df=0.75)
tfidf_vect_ngram_fit = tfidf_vect_ngram.fit_transform(data['clean_text'])
tfidf_ngram_array = tfidf_vect_ngram_fit.toarray()

In [24]:
tfidf_cols = ['tfidf_uni_'+str(i) for i in tfidf_vect.get_feature_names()]
tfidf_df = pd.DataFrame(tfidf_array,columns=tfidf_cols)

tfidf_ngram_cols = ['tfidf_ngram_'+str(i) for i in tfidf_vect_ngram.get_feature_names()]
tfidf_ngram_df = pd.DataFrame(tfidf_ngram_array,columns=tfidf_ngram_cols)

In [25]:
train = pd.concat([data,tfidf_df,tfidf_ngram_df],axis=1)
train.shape

(1000, 3303)

In [26]:
#def for add weights to columns 
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',np.unique(train.label),train.label))
weights =np.ones(train.shape[0],dtype='float')
for i,val in enumerate(train.label):
     weights[i]=class_weights[val]
train['weights_class']=weights

In [27]:
train_1=train.copy()
import random 
random.seed(20)


# Concatenating the columns which would be fed into the model as 'x' i.e. dependent
training_cols = tfidf_cols+tfidf_ngram_cols

weights='weights_class'

other_cols=['label','weights_class']

tk=other_cols+training_cols

response = 'label'
c=train[tk]
# Conversion of pandas dataframe to h2o dataframe
print(c.shape)
train_frame=h2o.H2OFrame(c)
print(train_frame.shape)

(1000, 3302)
Parse progress: |█████████████████████████████████████████████████████████| 100%
(1000, 3302)


In [28]:
train_frame['label'] = train_frame['label'].asfactor()
print(train_frame.shape)

(1000, 3302)


In [29]:
#Splitting the dataframe in train and val
train, valid= train_frame.split_frame(ratios = [.9], seed = 1234)
print(train.shape)
print(valid.shape)

(897, 3302)
(103, 3302)


In [30]:
#training_cols=pre_cols+tfidf_cols+tfidf_ngram_cols+ embedding_cols + columns_pos
from h2o.estimators import H2ORandomForestEstimator
model = H2ORandomForestEstimator(ntrees=50,max_depth=6,nfolds=5,balance_classes=False,
                                 keep_cross_validation_predictions=True,
                                 fold_assignment="stratified",
                                 keep_cross_validation_models=True,gainslift_bins=10,weights_column='weights_class',seed=10)
model.train(x=training_cols,y=response, training_frame=train,validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [31]:
with open('tfidf_pickle','wb') as f:
    pickle.dump(tfidf_vect,f)
with open('tfidfngram_pickle','wb') as f:
    pickle.dump(tfidf_vect_ngram,f)

In [32]:
# model_2.model_performance()
path = '/home/ec2-user/SageMaker/imdb_train_deploy'
h2o.save_model(model,path)

'/home/ec2-user/SageMaker/imdb_train_deploy/DRF_model_python_1613742661629_1'